This notebook provides a basic interface for creating a Flask application for recording airport checks. 
It is a simple procedure using pandas DataFrame with input data and folium for basemap and visualization.

In [1]:
import random
import pandas as pd
import folium
import numpy as np

from folium.plugins import MeasureControl, Search, Geocoder
from folium import FeatureGroup

In [5]:
#read csv file with open a fictional data file from Pardubice Airport (ICAO:LKPD)

air_tab = pd.read_csv("/content/Airport_app.csv", delimiter =';')

# generate random coordinates of controls of events 
# we used airport coordinates from 50.016544, 15.720564 to 50.012324, 15.759617

air_tab['Lon'] = np.random.randint(15720564, 15759617, air_tab.shape[0])
air_tab['Lon'] = air_tab['Lon']/1000000
air_tab['Lat'] = np.random.randint(50012324, 50016544, air_tab.shape[0])
air_tab['Lat'] = air_tab['Lat']/1000000
air_tab.head()

,ID,Date,Time,inspector ID,RWYCC A,RWYCC B,RWYCC C,RWY A,RWY B,RWY C,...,Friction B,Friction C,FOD/Failure,unpaved area,Arresting system A,Arresting system B,Chemically treated,Status,Lon,Lat
0,1,01.01.2022,8:00:00,1111,6,6,6,DRY,DRY,DRY,...,-,-,False,01.I,True,True,False,1,15.753242,50.015939
1,2,02.01.2022,9:00:00,1211,5,5,5,WET,WET,WET,...,-,-,False,01.I,True,True,False,0,15.722921,50.012363
2,3,03.01.2022,10:00:00,1311,5,5,5,WET,WET,WET,...,-,-,False,0.9,True,True,False,1,15.755296,50.016191
3,4,04.01.2022,11:00:00,1411,5,5,5,WET,WET,WET,...,-,-,False,0.8,True,True,False,1,15.724915,50.013778
4,5,05.01.2022,12:00:00,1511,4,4,4,COMPACTED SNOW,COMPACTED SNOW,COMPACTED SNOW,...,0.58,0.59,False,0.9,True,True,True,1,15.721253,50.016020


In [6]:
# UDF for creating layer of points retrieved from Series

def get_layer(m, df,series, nam, valu, color = 'blue', icon = 'circle-dot', add_search = False):
  ''' m for map,df for source Pandas, 
  series for name of series to use as str, nam for name of the layer as str
  icons and colors can be set, full list of icons available here https://fontawesome.com/search?s=solid&f=sharp&o=r
  valu is value to filter by'''

  # Create Empty Status Layer 
  status_layer = folium.FeatureGroup(name= nam)
  
  for index, row in air_tab.iterrows():
      if row[series] == valu:
        color = color
        # create a marker for the point
        marker = folium.Marker(location =[row['Lat'], row['Lon']],icon=folium.Icon(color=color, prefix='fa', icon=icon))
        # add the marker to the feature group
        marker.add_to(status_layer)

  # add the feature group to the map
  status_layer.add_to(m)
  
  # Searching window by
  if add_search:
    
    Geocoder().add_to(m)
    search = Search(
      layer=status_layer,  # the layer to search within
      geom_type='Point',  # the type of geometry to search for
      placeholder=f'Search in {nam} layer...',  # the placeholder text for the search box
      collapsed=False,  # whether the search box should be collapsed by default
      search_label='Inspector ID',  # the label for the search results
        )
   # add the Search plugin to the map
    search.add_to(m)
  return m

def Airport_T(airp):
    # Create a map
    m = folium.Map(location=[airp['Lat'].mean(), airp['Lon'].mean()], zoom_start=14)

    # Set Tiles with Satellite BaseMap
    tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
    
    # This part creates layers by values in Series of pandas
    # You can also use marker_cluster = MarkerCluster().add_to(m) to reduce number of points
    # Preferably, user's input would be requested beforehand via Form controls
    for index, row in air_tab.iterrows():
      dd = row['Date']
      get_layer(m,airp,'Date',str(dd), valu = dd, color = 'blue', icon='timer')

    # Calling function for three requested layers
    get_layer(m,airp,'RWY A','RWY A = DRY','DRY',add_search = True)  
    get_layer(m,airp,'Status','Resolved',1,color = 'green', icon='check') 
    get_layer(m,airp,'Status','Unsolved',0,color = 'red', icon='xmark') 



    folium.plugins.ScrollZoomToggler().add_to(m)
    folium.plugins.Fullscreen(position='topright').add_to(m)
    folium.plugins.MarkerCluster().add_to(m)

    # add the layer control to the map
    
    folium.LayerControl().add_to(m)

    m.add_child(MeasureControl())
    return m

In [7]:
# Call function and show map
Airport_T(air_tab)